In [1]:
import cantera as ct
import os 
import re
from rmgpy.molecule import Molecule
from rmgpy.reaction import Reaction
from rmgpy.species import Species
from rmgpy.data.kinetics.family import KineticsFamily
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings
from rmgpy.data.base import Database
from rmgpy.chemkin import load_species_dictionary

Make a species dictionary

In [2]:
unneeded_species = ['CHF(T)',
'CF2(T)',
'CH3CF(T)',
'CH2FCH(T)',
'CH2FCF(T)',
'CHF2CH(S)',
'CHF2CF(T)',
'CF3CH(S)',
'CF3CF(T)',
'CF3CO']


species_dictionary = {}


file = './hcof_anl0_with_c2_20231126.yaml'

with open(file, 'r') as f: 
    lines = f.readlines()
#species are in lines 12-103
for line in lines[11:103]:
    split_species = line.split(', #')
    [label, smiles] = split_species
    
    #handle the label first
    label = label.replace(' ', '')
        
    #now the smiles
    smiles = smiles.replace(' ', '').replace('\n', '').replace(',','')
    match=re.search('([a-z]+)', smiles)
    if match:
        also_remove = match.group(1)
        smiles = smiles.replace(also_remove, '')
        
    #now save to species dictionary
    if label in unneeded_species: 
        continue
    else:
        species_dictionary[label] = smiles
        

In [3]:
#add the last couple of stragglers

species_dictionary['H']= "[H]"
species_dictionary['H2']= "[H][H]"
species_dictionary['CH']= "[CH]"
species_dictionary['CH2']= "[CH2]"
species_dictionary['CH3']= "[CH3]"
species_dictionary['CH4']= "C"
species_dictionary['O']= "[O]"
species_dictionary['OH']= "[OH]"
species_dictionary['H2O']= "O"
species_dictionary['O2']= "[O][O]"
species_dictionary['HO2']= "[O]O"
species_dictionary['H2O2']= "OO"
species_dictionary['CO']= "[C-]#[O+]"
species_dictionary['CO2']= "O=C=O"
species_dictionary['HCO']= "[CH]=O"
species_dictionary['CH2O'] = "C=O"

Now let's make the reaction.py

But we need the degeneracy first

In [5]:
gas = ct.Solution('/work/westgroup/nora/Code/projects/PFAS/ESSCI/models/ANL_Brown5/format_reactions/F_abstraction/hcof_anl0_with_c2_20231126.yaml')

In [3]:
#load in the database
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = ['Klippenstein_Glarborg2016', 'BurkeH2O2', 'thermo_DFT_CCSDTF12_BAC', 
                               'DFT_QCI_thermo',
                           'primaryThermoLibrary', 'primaryNS', 'NitrogenCurran', 'NOx2018', 'FFCM1(-)',
'SulfurLibrary', 'SulfurGlarborgH2S','SABIC_aromatics'],
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = 'all',
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )

In [ ]:
#H-abstraction 0-40(including)
#F-abstraction 41-80
#Substitution 81, 82, 83, 84
#will have to make a Brown halogens pdep library too

In [38]:
rmg_family = 'F_Abstraction'
H_abstraction_lines = []
continued_count = 90 #90 was last index in H_abstraction reactions.py
for index, rxn in enumerate([gas.reactions()[16]]):
    continued_count+=1
 
    #get rate parameters
    A = "{:e}".format(rxn.rate.pre_exponential_factor*(100*100*100)/1000) #convert from m3 / kmol s to cm3 / mol s
    n = rxn.rate.temperature_exponent
    Ea = rxn.rate.activation_energy/(1000*1000) #convert from J/kmol to kJ/mol
    
    #get the degeneracy using RMG
    reactants = list(rxn.reactants.keys())
    products = list(rxn.products.keys())

    reactant_Molecule = [Molecule(smiles=species_dictionary[x]) for x in reactants]
    product_Molecule = [Molecule(smiles=species_dictionary[x]) for x in products]

    try: 
        template_reaction = database.kinetics.families[rmg_family].generate_reactions(reactants=reactant_Molecule, products=product_Molecule)[0] #just choose the first
        degeneracy = database.kinetics.families[rmg_family].calculate_degeneracy(template_reaction)
    except IndexError as e:
        print(index, rxn)
        print(e)
        continue
    
    #now match to species already existing in reactions.py
    labeled_adjacencies = get_labeled_adj_lists(rxn)
    switch_labels = get_matching_species_label(labeled_adjacencies)

    
    #and you should change the labels to existing labels in the reactions.py file if the species already exists there 
    final_reactants = []
    for reactant in reactants: 
        try:
            final_label = switch_labels[reactant]
        except KeyError as e: 
            print(index, rxn)
            print(e)
            final_label = reactant
        final_reactants.append(final_label)

    equation_string = ''
    for reactant in final_reactants: 
        equation_string+=f'{reactant} + '
    equation_string=equation_string[:-3] + ' <=> ' #remove the last bit when the loop finishes and add a '<=>'


    final_products = []
    for product in products: 
        try:
            final_label = switch_labels[product]
        except KeyError as e: 
            final_label = product
        final_products.append(final_label)

    #now add to the equation string
    for product in final_products: 
        equation_string+=f'{product} + '
    equation_string=equation_string[:-3] #remove the last bit when the loop finishes and add a '<=>'

    f_string = f'''
entry(
    index = {continued_count},
    label = "{equation_string}",
    degeneracy = {degeneracy},
    kinetics = Arrhenius(A=({A},'cm^3/(mol*s)'), n={n}, Ea=({Ea},'kJ/mol'), T0=(1,'K'), Tmin=(300,'K'), Tmax=(2000,'K')),
    rank = 5,
    shortDesc = """ANL0 method from Brown""",
    longDesc = 
"""
Electronic structures done using ANL0 compound method. 
Torsional scans with  M06-2X/cc-pVTZ. 
""",
)
    '''
    print(f_string)

0 CF3 + FOH <=> CHF3 + OF
list index out of range


In [9]:
#existing F abstraction training dictionary 
e_path = '/home/khalil.nor/Code/RMG-database/input/kinetics/families/F_Abstraction/training/dictionary.txt'
existing_dict = load_species_dictionary(e_path)

In [23]:

def get_labeled_adj_lists(rxn): 
    """
    given a reaction, use RMG's .get_labeled_reactants_and_products to get the actual adjacency lists with all the *#s 

    """
    
    labeled_adjacencies = {}

    reactants = list(rxn.reactants.keys())
    products = list(rxn.products.keys())

    #make all Molecule objects
    reactant_Molecule = [Molecule(smiles=species_dictionary[x]) for x in reactants]
    product_Molecule = [Molecule(smiles=species_dictionary[x]) for x in products]

    reactant_Species = [Species(molecule=[Molecule(smiles=species_dictionary[x])], label=x) for x in reactants]
    product_Species = [Species(molecule=[Molecule(smiles=species_dictionary[x])], label=x) for x in products]


    generated_labels = database.kinetics.families[rmg_family].get_labeled_reactants_and_products(reactant_Molecule,product_Molecule, relabel_atoms=True)

    #reactants
    for reactant_s in reactant_Species: 
        for reactant in generated_labels[0]: #generated_labels[0] is list of reactants from .get_labeled_reactants_and_products
            if reactant.is_isomorphic(reactant_s.molecule[0]):  #let's match which generated label goes with which reactant
                reactant_label = reactant_s.label
                if reactant_label not in labeled_adjacencies.keys():
                    labeled_adjacencies[reactant_label]=[reactant.to_adjacency_list(), reactant_s]

    #products 
    for product_s in product_Species: 
        for product in generated_labels[1]: 
            if product.is_isomorphic(product_s.molecule[0]):
                product_label = product_s.label
                if product_label not in labeled_adjacencies.keys():
                    labeled_adjacencies[product_label]=[product.to_adjacency_list(), product_s]
    
    return labeled_adjacencies #labeled_adjacencies is a dictionary with key = species label, value = [generated adj list, Species object]

def get_matching_species_label(labeled_adjacencies):   

    switch_labels = {}
    for key, value in labeled_adjacencies.items():
        [adj_list, spec] = value
        starred = re.findall('(\*[0-9] [A-Z])', adj_list)
        for label, species in existing_dict.items(): #let's see if theres an existing species name that precisely matches this adj list
            if species.is_isomorphic(spec):
                flag = 0 
                for starred_str in starred:
                    test_str = species.to_adjacency_list()
                    if starred_str in test_str: #that there is an existing species in dictionary.txt that has a "*# Atom_symbol"
                        pass
                    else: 
                        flag+=1
                        
                    #make sure theres no extra stars in the existing species adj list that we're looking at 
                    already_present = re.findall('(\*[0-9] [A-Z])', test_str)
                    if len(already_present)>len(starred):
                        #this means theres an extra
                        flag+=1
                if flag == 0:
                    switch_labels[key]=label #if everything has worked out...
                    # print(species.to_adjacency_list())
                    # print('matched to')
                    # print(adj_list)

    return switch_labels

In [6]:
#template_reaction = database.kinetics.families[rmg_family].generate_reactions(reactants=reactant_Molecule, products=product_Molecule)[0] #just choose the first
rmg_family = 'F_Abstraction'
x = database.kinetics.families[rmg_family].get_training_set()


In [8]:
for ind, y in enumerate(x):
    print(ind, y)

0 F2 + CF3 <=> CF4_p23 + F_p1
1 CH3 + F2 <=> CH3F_p23 + F_p1
2 [OH]_r3 + FC(Cl)(Cl)Cl_r12 <=> Cl[C](Cl)Cl_p1 + OF_p23
3 [OH]_r3 + [CH2]F_r12 <=> [CH2]_p1 + OF_p23
4 OOF_r12 + CH3 <=> [O]O_p1 + CH3F_p23
5 C[C](C)F_r12 + CH3 <=> C[C]C_p1 + CH3F_p23
6 CDC(C)F_r12 + CH3 <=> CD[C]C_p1 + CH3F_p23
7 CDCF_r12 + CH3 <=> [CH]DC_p1 + CH3F_p23
8 COF_r12 + CH3 <=> C[O]_p1 + CH3F_p23
9 OD[C]F_r12 + CH3 <=> [C]DO_p1 + CH3F_p23
10 [OH]_r3 + FCCl_r12 <=> [CH2]Cl_p1 + OF_p23
11 CC(C)F_r12 + CH3 <=> C[CH]C_p1 + CH3F_p23
12 [O]_r3 + CDC(C)F_r12 <=> CD[C]C_p1 + [O]F_p23
13 O[CH]F_r12 + CH3 <=> [CH]O_p1 + CH3F_p23
14 [O]_r3 + O[CH]F_r12 <=> [CH]O_p1 + [O]F_p23
15 CCF_r12 + CH3 <=> C[CH2]_p1 + CH3F_p23
16 C[CH]F_r12 + CH3 <=> [CH]C_p1 + CH3F_p23
17 [O]O_r3 + [O]F_r12 <=> [O]_p1 + OOF_p23
18 OCF_r12 + CH3 <=> [CH2]O_p1 + CH3F_p23
19 [O]O_r3 + O[CH]F_r12 <=> [CH]O_p1 + OOF_p23
20 [OH]_r3 + CDCDCF_r12 <=> [CH]DCDC_p1 + OF_p23
21 [OH]_r3 + C[C](C)F_r12 <=> C[C]C_p1 + OF_p23
22 C[C](O)F_r12 + CH3 <=> C[C]O_p1 + C

In [9]:
new_reactions = []
for ind, y in enumerate(x):
     if ind > 89:
            new_reactions.append(y)

In [11]:
old_reactions = []
for ind, y in enumerate(x):
     if ind <= 89:
            old_reactions.append(y)

In [12]:
keep = []
toss = []
for old_rxn in old_reactions:
    flag=0
    for new_rxn in new_reactions:
        if old_rxn.is_isomorphic(new_rxn):
            print(old_rxn, new_rxn)
            toss.append(old_rxn)
            flag+=1            
    if flag==0:
        keep.append(old_rxn)
        

F2 + CF3 <=> CF4_p23 + F_p1 CF3 + F2 <=> CF4 + F
CH3 + F2 <=> CH3F_p23 + F_p1 CH3 + F2 <=> CH3F + F
CH2F2 + CHO <=> CHFO + CH2F CH2F2 + CHO <=> CH2F + CHFO
CH3F + H <=> HF + CH3 CH3F + H <=> CH3 + HF
CH2F2 + H <=> HF + CH2F CH2F2 + H <=> CH2F + HF
CH3F + CH2F <=> CH2F2 + CH3 CH2F + CH3F <=> CH2F2 + CH3
CHF3 + H <=> HF + CHF2 CHF3 + H <=> CHF2 + HF
CH3F + CHF2 <=> CHF3 + CH3 CH3F + CHF2 <=> CH3 + CHF3
CH3F + CF3 <=> CF4 + CH3 CF3 + CH3F <=> CF4 + CH3
CF4 + H <=> HF + CF3 CF4 + H <=> CF3 + HF
CH3F + CHO <=> CHFO + CH3 CH3F + CHO <=> CH3 + CHFO
CF2O + H <=> HF + CFO CF2O + H <=> CFO + HF


In [13]:
for x in toss:
    print(x)

F2 + CF3 <=> CF4_p23 + F_p1
CH3 + F2 <=> CH3F_p23 + F_p1
CH2F2 + CHO <=> CHFO + CH2F
CH3F + H <=> HF + CH3
CH2F2 + H <=> HF + CH2F
CH3F + CH2F <=> CH2F2 + CH3
CHF3 + H <=> HF + CHF2
CH3F + CHF2 <=> CHF3 + CH3
CH3F + CF3 <=> CF4 + CH3
CF4 + H <=> HF + CF3
CH3F + CHO <=> CHFO + CH3
CF2O + H <=> HF + CFO


In [ ]:
#F2 + CF3 <=> CF4_p23 + F_p1
#CH3 + F2 <=> CH3F_p23 + F_p1
#CH2F2 + CHO <=> CHFO + CH2F
#CH3F + H <=> HF + CH3
#CH2F2 + H <=> HF + CH2F
#CH3F + CH2F <=> CH2F2 + CH3
#CHF3 + H <=> HF + CHF2
#CH3F + CHF2 <=> CHF3 + CH3
#CH3F + CF3 <=> CF4 + CH3
#CF4 + H <=> HF + CF3
#CH3F + CHO <=> CHFO + CH3
#CF2O + H <=> HF + CFO